## Regime Switching for Pairs Trading

When trading a spread between two correlated assets (like BTC and ETH), the key question is:
**when should I enter and exit positions?**

Let's look at real data to see why this is tricky...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fetch_data import fetch_btc_eth_history, estimate_ou_parameters

# Fetch 30 days of hourly BTC/ETH data
df = fetch_btc_eth_history(days=30, interval="1h")
spread = df["log_ratio"].values
dt = 1 / 24  # hourly data = 1/24 of a day

# Estimate OU parameters (mean-reversion model)
params = estimate_ou_parameters(spread, dt)
mu, sigma = params["mu"], params["sigma"]

print(f"Mean (μ): {mu:.4f}")
print(f"Std dev (σ): {sigma:.4f}")
print(f"Half-life: {params['half_life']*24:.1f} hours")

# Visualize the spread with mean and standard deviation bands
plt.style.use("dark_background")
fig, ax = plt.subplots(figsize=(12, 5))

# Plot the spread
times = df["open_time"]
ax.plot(times, spread, color="cyan", linewidth=1.2, label="BTC/ETH Spread (log ratio)")

# Mean line
ax.axhline(mu, color="white", linestyle="--", linewidth=1.5, label=f"Mean (μ = {mu:.3f})")

# ±1σ bands (mild opportunity zone)
ax.fill_between(times, mu - sigma, mu + sigma, color="green", alpha=0.25, label="±1σ band")

# ±2σ bands (strong opportunity zone)
ax.fill_between(times, mu - 2*sigma, mu - sigma, color="red", alpha=0.25, label="±2σ band")
ax.fill_between(times, mu + sigma, mu + 2*sigma, color="red", alpha=0.25)

# Styling
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("log(BTC/ETH)", fontsize=12)
ax.set_title("BTC/ETH Spread: Where Would You Enter?", fontsize=14, fontweight="bold")
ax.legend(loc="upper right", fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Paper Simplfied
https://arxiv.org/abs/2512.04697v1

## Model training data

## Run simulations

## Train on simulated Data

## Coming back to pairs trading

## Backtesting